In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import torch 
import json
import sys
import torch
import torch.nn.functional as F
import os
import numpy as np

sys.path.append("src")
os.chdir('/home/george-vengrovski/Documents/projects/tweety_bert_paper')

from data_class import SongDataSet_Image, CollateFunction
from model import TweetyBERT
from analysis import plot_umap_projection
from utils import detailed_count_parameters, load_weights, load_model

/home/george-vengrovski/anaconda3/envs/tweetybert/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/george-vengrovski/anaconda3/envs/tweetybert/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/george-vengrovski/anaconda3/envs/t

In [ ]:
from torch.utils.data import DataLoader
from data_class import SongDataSet_Image

train_dir = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/llb3_train"
test_dir = "/home/george-vengrovski/Documents/projects/tweety_bert_paper/files/llb3_test"

train_dataset = SongDataSet_Image(train_dir, num_classes=21)
test_dataset = SongDataSet_Image(test_dir, num_classes=21)

collate_fn = CollateFunction(segment_length=1000)  # Adjust the segment length if needed

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

In [3]:
from linear_probe import LinearProbeModel, LinearProbeTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier_model = LinearProbeModel(num_classes=21, model_type="raw", model=None, freeze_layers=None, layer_num=None, layer_id=None, classifier_dims=196)
classifier_model = classifier_model.to(device)

In [4]:
trainer = LinearProbeTrainer(model=classifier_model, train_loader=train_loader, test_loader=test_loader, device=device, lr=1e-3, plotting=True, batches_per_eval=100, desired_total_batches=1e4, patience=4)
trainer.train()

Batch 100: FER = 51.91%, Train Loss = 2.4177, Val Loss = 2.3121
Batch 200: FER = 51.58%, Train Loss = 2.0817, Val Loss = 2.0814
Batch 300: FER = 50.32%, Train Loss = 1.8640, Val Loss = 1.9755
Batch 400: FER = 49.89%, Train Loss = 1.9262, Val Loss = 1.9379
Batch 500: FER = 49.49%, Train Loss = 2.2262, Val Loss = 1.9562
Batch 600: FER = 49.70%, Train Loss = 1.7740, Val Loss = 1.9438
Batch 700: FER = 51.65%, Train Loss = 1.8729, Val Loss = 1.9594
Batch 800: FER = 50.11%, Train Loss = 1.8577, Val Loss = 1.9438
Batch 900: FER = 51.46%, Train Loss = 1.8825, Val Loss = 1.9663
Batch 1000: FER = 51.96%, Train Loss = 1.7255, Val Loss = 1.9576
Batch 1100: FER = 49.79%, Train Loss = 1.7670, Val Loss = 1.9778


KeyboardInterrupt: 

In [5]:
from linear_probe import ModelEvaluator

evaluator = ModelEvaluator(classifier_model, test_loader, use_tqdm=True)
class_frame_error_rates, total_frame_error_rate = evaluator.validate_model_multiple_passes(num_passes=1, max_batches=1250)
evaluator.save_results(class_frame_error_rates, total_frame_error_rate, 'results')

Evaluating: 100%|██████████| 142/142 [00:03<00:00, 43.22batch/s]
